In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


# 베이스 경로 설정
BASE_DIR = '/content/drive/MyDrive/Dacon_FakeText/'
SAVE_PATH = BASE_DIR + 'data/embeddings/train_concat.npy'

Mounted at /content/drive


In [ ]:
!pip install iterative-stratification

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tqdm import tqdm
import os

In [ ]:
# ✅ 1. 설정값
n_splits = 5
batch_size = 512
epochs = 30
early_stopping_rounds = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# ✅ 2. 데이터 로딩
X_train = np.load(BASE_DIR + 'data/embeddings/train_concat.npy')
train_meta = pd.read_csv(BASE_DIR + 'data/train_paragraph.csv')
y_train = train_meta['generated'].values
groups = train_meta['title'].values

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (1226364, 775), y_train shape: (1226364,)


In [ ]:
# ✅ 3. PyTorch Dataset 클래스
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.eps = 1e-8  # log 안정성 확보용

    def forward(self, inputs, targets):
        # inputs: sigmoid 통과된 예측 (batch_size, 1)
        # targets: (batch_size, 1)
        BCE = -(targets * torch.log(inputs + self.eps) + (1 - targets) * torch.log(1 - inputs + self.eps))
        focal = self.alpha * (1 - inputs)**self.gamma * targets * BCE + \
                (1 - self.alpha) * inputs**self.gamma * (1 - targets) * BCE
        return focal.mean()

In [ ]:
class StratifiedGroupKFold:
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y, groups):
        # 그룹을 정수로 인코딩
        if not np.issubdtype(groups.dtype, np.number):
            groups = LabelEncoder().fit_transform(groups)

        # 그룹별 인덱스 저장
        group_to_indices = {}
        for idx, g in enumerate(groups):
            group_to_indices.setdefault(g, []).append(idx)

        unique_groups = np.array(list(group_to_indices.keys()))
        group_y = np.array([
            int(y[group_to_indices[g]].mean() >= 0.5) for g in unique_groups
        ])

        skf = StratifiedKFold(
            n_splits=self.n_splits,
            shuffle=self.shuffle,
            random_state=self.random_state
        )

        for group_train_idx, group_val_idx in skf.split(unique_groups, group_y):
            train_indices, val_indices = [], []

            for gi in group_train_idx:
                train_indices.extend(group_to_indices[unique_groups[gi]])
            for gi in group_val_idx:
                val_indices.extend(group_to_indices[unique_groups[gi]])

            yield np.array(train_indices), np.array(val_indices)

In [ ]:
# 4. MLP 모델 클래스
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# ✅ 5. GroupKFold + 학습 + 로그 저장
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X_train))

all_logs = []  # 전체 로그 저장용 리스트
y_train=y_train.astype(np.float32)
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train, y_train, groups)):
    print(f'\n=== Fold {fold+1}/{n_splits} 학습 시작 ===')

    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]

    train_dataset = CustomDataset(X_tr, y_tr)
    val_dataset = CustomDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = MLP(input_dim=X_train.shape[1]).to(device)
    criterion = FocalLoss(alpha=0.25, gamma=2.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',           # val_auc가 증가해야 하므로 'max'
    factor=0.5,           # lr을 절반으로 줄임
    patience=3,           # 2 epoch 동안 개선 없으면 감소
    verbose=True,
    min_lr=1e-5           # 최소 학습률 하한
    )

    best_auc = 0
    patience = 0
    fold_log = []  # 이 fold의 epoch별 로그 저장용 리스트

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb.view(-1,1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_preds = []
        with torch.no_grad():
            for xb, _ in val_loader:
                xb = xb.to(device)
                pred = model(xb).squeeze()
                pred = torch.sigmoid(pred)
                val_preds.extend(pred.cpu().numpy())

        auc = roc_auc_score(y_val, val_preds)
        avg_train_loss = train_loss / len(train_loader)

        print(f"Fold {fold+1} | Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val AUC: {auc:.4f}")

        # LR Scheduler 업데이트
        scheduler.step(auc)

        # ✅ 로그 저장
        fold_log.append({
            'fold': fold + 1,
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'val_auc': auc
        })

        # Early Stopping
        if auc > best_auc:
            best_auc = auc
            patience = 0
            torch.save(model.state_dict(), BASE_DIR + f'model/mlp_fold{fold+1}.pt')
            print(f"Fold {fold+1} 모델 저장 (Best AUC: {best_auc:.4f})")
        else:
            patience += 1
            if patience >= early_stopping_rounds:
                print(f"Early Stopping (patience {early_stopping_rounds} 도달)")
                break

    # Fold별 로그 저장
    all_logs.extend(fold_log)

    # Fold OOF
    oof_preds[val_idx] = val_preds




=== Fold 1/5 학습 시작 ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Fold 1 | Epoch 1 | Train Loss: 0.0273 | Val AUC: 0.6921
✅ Fold 1 모델 저장 (Best AUC: 0.6921)
Fold 1 | Epoch 2 | Train Loss: 0.0231 | Val AUC: 0.7047
✅ Fold 1 모델 저장 (Best AUC: 0.7047)
Fold 1 | Epoch 3 | Train Loss: 0.0223 | Val AUC: 0.7103
✅ Fold 1 모델 저장 (Best AUC: 0.7103)
Fold 1 | Epoch 4 | Train Loss: 0.0220 | Val AUC: 0.7159
✅ Fold 1 모델 저장 (Best AUC: 0.7159)
Fold 1 | Epoch 5 | Train Loss: 0.0218 | Val AUC: 0.7175
✅ Fold 1 모델 저장 (Best AUC: 0.7175)
Fold 1 | Epoch 6 | Train Loss: 0.0217 | Val AUC: 0.7181
✅ Fold 1 모델 저장 (Best AUC: 0.7181)
Fold 1 | Epoch 7 | Train Loss: 0.0217 | Val AUC: 0.7194
✅ Fold 1 모델 저장 (Best AUC: 0.7194)
Fold 1 | Epoch 8 | Train Loss: 0.0216 | Val AUC: 0.7216
✅ Fold 1 모델 저장 (Best AUC: 0.7216)
Fold 1 | Epoch 9 | Train Loss: 0.0215 | Val AUC: 0.7221
✅ Fold 1 모델 저장 (Best AUC: 0.7221)
Fold 1 | Epoch 10 | Train Loss: 0.0215 | Val AUC: 0.7223
✅ Fold 1 모델 저장 (Best AUC: 0.7223)
Fold 1 | Epoch 11 | Train Loss: 0.0214 | Val AUC: 0.7248
✅ Fold 1 모델 저장 (Best AUC: 0.7248)
Fold 1 |

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Fold 2 | Epoch 1 | Train Loss: 0.0277 | Val AUC: 0.6877
✅ Fold 2 모델 저장 (Best AUC: 0.6877)
Fold 2 | Epoch 2 | Train Loss: 0.0234 | Val AUC: 0.7002
✅ Fold 2 모델 저장 (Best AUC: 0.7002)
Fold 2 | Epoch 3 | Train Loss: 0.0225 | Val AUC: 0.7076
✅ Fold 2 모델 저장 (Best AUC: 0.7076)
Fold 2 | Epoch 4 | Train Loss: 0.0221 | Val AUC: 0.7106
✅ Fold 2 모델 저장 (Best AUC: 0.7106)
Fold 2 | Epoch 5 | Train Loss: 0.0220 | Val AUC: 0.7148
✅ Fold 2 모델 저장 (Best AUC: 0.7148)
Fold 2 | Epoch 6 | Train Loss: 0.0218 | Val AUC: 0.7171
✅ Fold 2 모델 저장 (Best AUC: 0.7171)
Fold 2 | Epoch 7 | Train Loss: 0.0217 | Val AUC: 0.7184
✅ Fold 2 모델 저장 (Best AUC: 0.7184)
Fold 2 | Epoch 8 | Train Loss: 0.0217 | Val AUC: 0.7196
✅ Fold 2 모델 저장 (Best AUC: 0.7196)
Fold 2 | Epoch 9 | Train Loss: 0.0216 | Val AUC: 0.7217
✅ Fold 2 모델 저장 (Best AUC: 0.7217)
Fold 2 | Epoch 10 | Train Loss: 0.0216 | Val AUC: 0.7183
Fold 2 | Epoch 11 | Train Loss: 0.0215 | Val AUC: 0.7223
✅ Fold 2 모델 저장 (Best AUC: 0.7223)
Fold 2 | Epoch 12 | Train Loss: 0.0215 | V

...

Fold 3 | Epoch 1 | Train Loss: 0.0277 | Val AUC: 0.6963
✅ Fold 3 모델 저장 (Best AUC: 0.6963)
Fold 3 | Epoch 2 | Train Loss: 0.0234 | Val AUC: 0.7118
✅ Fold 3 모델 저장 (Best AUC: 0.7118)
Fold 3 | Epoch 3 | Train Loss: 0.0224 | Val AUC: 0.7156
✅ Fold 3 모델 저장 (Best AUC: 0.7156)
Fold 3 | Epoch 4 | Train Loss: 0.0222 | Val AUC: 0.7188
✅ Fold 3 모델 저장 (Best AUC: 0.7188)
Fold 3 | Epoch 5 | Train Loss: 0.0220 | Val AUC: 0.7235
✅ Fold 3 모델 저장 (Best AUC: 0.7235)
Fold 3 | Epoch 6 | Train Loss: 0.0219 | Val AUC: 0.7252
✅ Fold 3 모델 저장 (Best AUC: 0.7252)
Fold 3 | Epoch 7 | Train Loss: 0.0218 | Val AUC: 0.7253
✅ Fold 3 모델 저장 (Best AUC: 0.7253)
Fold 3 | Epoch 8 | Train Loss: 0.0217 | Val AUC: 0.7266
✅ Fold 3 모델 저장 (Best AUC: 0.7266)
Fold 3 | Epoch 9 | Train Loss: 0.0217 | Val AUC: 0.7301
✅ Fold 3 모델 저장 (Best AUC: 0.7301)
Fold 3 | Epoch 10 | Train Loss: 0.0216 | Val AUC: 0.7266
Fold 3 | Epoch 11 | Train Loss: 0.0216 | Val AUC: 0.7332
✅ Fold 3 모델 저장 (Best AUC: 0.7332)
Fold 3 | Epoch 12 | Train Loss: 0.0216 | V

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Fold 4 | Epoch 1 | Train Loss: 0.0272 | Val AUC: 0.6991
✅ Fold 4 모델 저장 (Best AUC: 0.6991)
Fold 4 | Epoch 2 | Train Loss: 0.0231 | Val AUC: 0.7020
✅ Fold 4 모델 저장 (Best AUC: 0.7020)
Fold 4 | Epoch 3 | Train Loss: 0.0223 | Val AUC: 0.7128
✅ Fold 4 모델 저장 (Best AUC: 0.7128)
Fold 4 | Epoch 4 | Train Loss: 0.0220 | Val AUC: 0.7169
✅ Fold 4 모델 저장 (Best AUC: 0.7169)
Fold 4 | Epoch 5 | Train Loss: 0.0219 | Val AUC: 0.7204
✅ Fold 4 모델 저장 (Best AUC: 0.7204)
Fold 4 | Epoch 6 | Train Loss: 0.0217 | Val AUC: 0.7185
Fold 4 | Epoch 7 | Train Loss: 0.0216 | Val AUC: 0.7237
✅ Fold 4 모델 저장 (Best AUC: 0.7237)
Fold 4 | Epoch 8 | Train Loss: 0.0216 | Val AUC: 0.7207
Fold 4 | Epoch 9 | Train Loss: 0.0215 | Val AUC: 0.7299
✅ Fold 4 모델 저장 (Best AUC: 0.7299)
Fold 4 | Epoch 10 | Train Loss: 0.0214 | Val AUC: 0.7317
✅ Fold 4 모델 저장 (Best AUC: 0.7317)
Fold 4 | Epoch 11 | Train Loss: 0.0214 | Val AUC: 0.7242
Fold 4 | Epoch 12 | Train Loss: 0.0214 | Val AUC: 0.7268
Fold 4 | Epoch 13 | Train Loss: 0.0213 | Val AUC: 0.7

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Fold 5 | Epoch 1 | Train Loss: 0.0269 | Val AUC: 0.6971
✅ Fold 5 모델 저장 (Best AUC: 0.6971)
Fold 5 | Epoch 2 | Train Loss: 0.0229 | Val AUC: 0.7117
✅ Fold 5 모델 저장 (Best AUC: 0.7117)
Fold 5 | Epoch 3 | Train Loss: 0.0221 | Val AUC: 0.7136
✅ Fold 5 모델 저장 (Best AUC: 0.7136)
Fold 5 | Epoch 4 | Train Loss: 0.0219 | Val AUC: 0.7218
✅ Fold 5 모델 저장 (Best AUC: 0.7218)
Fold 5 | Epoch 5 | Train Loss: 0.0217 | Val AUC: 0.7219
✅ Fold 5 모델 저장 (Best AUC: 0.7219)
Fold 5 | Epoch 6 | Train Loss: 0.0216 | Val AUC: 0.7262
✅ Fold 5 모델 저장 (Best AUC: 0.7262)
Fold 5 | Epoch 7 | Train Loss: 0.0215 | Val AUC: 0.7278
✅ Fold 5 모델 저장 (Best AUC: 0.7278)
Fold 5 | Epoch 8 | Train Loss: 0.0215 | Val AUC: 0.7266
Fold 5 | Epoch 9 | Train Loss: 0.0213 | Val AUC: 0.7264
Fold 5 | Epoch 10 | Train Loss: 0.0213 | Val AUC: 0.7287
✅ Fold 5 모델 저장 (Best AUC: 0.7287)
Fold 5 | Epoch 11 | Train Loss: 0.0213 | Val AUC: 0.7334
✅ Fold 5 모델 저장 (Best AUC: 0.7334)
Fold 5 | Epoch 12 | Train Loss: 0.0212 | Val AUC: 0.7312
Fold 5 | Epoch 13 |

In [ ]:
# 6. 전체 OOF AUC
final_auc = roc_auc_score(y_train, oof_preds)
print(f"\n전체 OOF AUC: {final_auc:.4f}")

# 6-1. 전체 OOF 결과를 로그에 추가
log_df = pd.DataFrame(all_logs)
log_df = pd.concat([
    log_df,
    pd.DataFrame([{
        'fold': 0,
        'epoch': 0,
        'train_loss': None,
        'val_auc': final_auc
    }])
], ignore_index=True)

# 7. 최종 로그 CSV 저장
log_df.to_csv(BASE_DIR + 'logs/deep_mlp_training_log_3.csv', index=False, encoding='utf-8')
print(f"전체 학습 로그 저장 완료: {BASE_DIR}logs/deep_mlp_training_log_3.csv")


✅ 전체 OOF AUC: 0.7263


/tmp/ipython-input-15-2918466026.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  log_df = pd.concat([


✅ 전체 학습 로그 저장 완료: /content/drive/MyDrive/Dacon_FakeText/logs/deep_mlp_training_log_3.csv
